# Housing Pipeline

In [1]:
import pandas as pd
import numpy as np

# https://numpy.org/doc/stable/reference/arrays.scalars.html
# https://numpy.org/doc/stable/reference/arrays.dtypes.html
# https://www.bbc.co.uk/bitesize/guides/zscvxfr/revision/3

# I'm type casting the data to save space in memory.
# Although the full CSV it's just ~1MB, it's generally useful to set the right type for each attribute.
housing = pd.read_csv(
    'housing.csv',
    dtype={
        'longitude': np.float32,
        'latitude': np.float32,
        'housing_median_age': np.uint8,
        'population': np.uint16,
        'households': np.uint16,
        'median_income': np.float32,
        'median_house_value': np.float32,
        'ocean_proximity': 'category'
    }
)

housing = housing.assign(
    # used to create stratified sampling for the training and test set
    median_income_categories = pd.cut(
        housing['median_income'],
        bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
        labels=[1, 2, 3, 4, 5]
    )
)

In [2]:
housing.isna().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms              207
population                    0
households                    0
median_income                 0
median_house_value            0
ocean_proximity               0
median_income_categories      0
dtype: int64

## Training, Validation and Test split

Split the original dataset in 3 subsets in order to generate the Training, Validation and Test sets, stratifying the samples baed on the `median_income_categories` attribute.

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for training_index, test_index in split.split(housing, housing['median_income_categories']):
    training_set = housing.loc[training_index]
    test_set = housing.loc[test_index]

Separate the label attribute from the rest of the dataset

In [4]:
X_training = training_set.drop('median_house_value', axis=1)
y_training = training_set[['median_house_value']].copy()

X_test = test_set.drop('median_house_value', axis=1)
y_test = test_set[['median_house_value']].copy()

## Pipeline

Create a custom transformer

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['rooms_per_household'] = X['total_rooms'] / X['households']
        X['bedrooms_per_room'] = X['total_bedrooms'] / X['total_rooms']
        X['population_per_household'] = X['population'] / X['households']
        return X

List numerical and categorical attribute names

In [6]:
label = 'median_house_value'

num_attributes = np.array(housing.drop([
    'ocean_proximity', # categorical
    'median_income_categories', # categorical
    label
], axis=1).columns)

num_attributes = np.append(num_attributes, ['rooms_per_household', 'bedrooms_per_room', 'population_per_household'])

cat_attributes = ['ocean_proximity']

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

column_transformer = ColumnTransformer([
    ('numerical', num_pipeline, num_attributes),
    ('categorical', cat_pipeline, cat_attributes),
])

pipeline = Pipeline([
    ('feature_engineering', FeatureEngineering()),
    ('column_transformation', column_transformer),
    ('regression', LinearRegression())
])

Run the pipeline with the **Training** set

In [8]:
pipeline.fit(X_training, y_training)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('feature_engineering', ...), ('column_transformation', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('numerical', ...), ('categorical', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If t

Fit and predict with **Linear Regression**

In [9]:
pipeline.score(X_training, y_training)

0.6529270339944889

In [10]:
pipeline.score(X_test, y_test)

0.6575916742330127

In [11]:
pipeline.named_steps.regression.coef_

array([[-56046.34122129, -56657.82988964,  13931.56896384,
          7996.06344394,    322.8068387 , -46114.16221575,
         42894.5027431 ,  78265.15986717,   7278.00688542,
         16859.18583961,    913.38710574, -17551.71144622,
        -51777.56425304, 105745.01327537, -22255.09084294,
        -14160.64673317]])

In [12]:
pipeline.named_steps.regression.intercept_

array([235539.00840088])

In [13]:
predictions = pipeline.predict(X_test)

In [14]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_test, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

np.float64(66803.13706652567)